In [1]:
import pandas as pd

def isNaN(string):
    return string != string

ms_file_handler = open(r"C:/Users/Admin/24[7]_Project/processed_text/marital_status_ds.txt","r", encoding="utf-8")

pre_doc_sentences = []
classes = []

for lines in ms_file_handler:
    sentences = lines.split("++$++")
    #print(sentences[0])
    pre_doc_sentences.append(sentences[0].lower())
    classes.append(int(sentences[1]))

In [2]:
categories=["Single", "Married"]

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 


def remove_punct(strin):
    punct_str = '!"#$%&\'.()*+,-/:;?@[\]^_`{|}~'
    restr = ""
    for letter in strin:
        if(letter not in punct_str):
            restr = restr+letter
        else:
            restr = restr+" "
    return restr

In [3]:
ind = 0

docs = []
y_class = []

for sent in pre_doc_sentences:
    clean_data = remove_punct(sent)
    actual_words=""
    for word in clean_data.split():
        if word.lower() not in stop_words and len(word) > 1:
            actual_words = actual_words + word.lower() + " "  
    #print("ind: ",ind,"  len_actual: ",len(actual_words.split()))
    if(len(actual_words.split())>50):
        temp_actual = actual_words.split()
        while (len(temp_actual)>50):
            sentence = ""
            temp_list = temp_actual[:50]
            for temp_words in temp_list:
                sentence += temp_words + " "
            temp_actual = temp_actual[50:]
            docs.append(sentence)
            y_class.append(classes[ind])
        if(len(temp_actual)>0):
            sentence = ""
            temp_list = temp_actual[:50]
            for temp_words in temp_list:
                sentence += temp_words + " "
            docs.append(sentence)
            y_class.append(classes[ind])
    else:
        docs.append(actual_words)
        y_class.append(classes[ind])
    ind += 1
#docs[0]

In [4]:
import tensorflow as tffrom 
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv1D
from tensorflow.keras.layers import Embedding

print("Assigning labels to fit in CNN...")

labels = np.array(y_class)
vocab_size = 2000

print("Encoding given words to values...")

encoded_docs = [one_hot(d, vocab_size) for d in docs]
#print(encoded_docs)
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
print("Defining the CNN model. Summary is provided below")
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Dropout(0.2))
model.add(Conv1D(filters=1,kernel_size=3,padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
print("Fitting the model with data...")
model.fit(padded_docs, labels, epochs=100, verbose=1)
# evaluate the model
print("Calculating accuracy...")
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Assigning labels to fit in CNN...
Encoding given words to values...
Defining the CNN model. Summary is provided below
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 8)             16000     
_________________________________________________________________
dropout (Dropout)            (None, 50, 8)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 48, 1)             25        
_________________________________________________________________
flatten (Flatten)            (None, 48)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 49        
Total params: 16,074
Trainable params: 16,074
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.save('marital_status_cnn_model.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: marital_status_cnn_model.model\assets


In [10]:
docs=['My boyfriend is a talented photographer working in Stanford university']
vocab_size = 2000
loaded_model = tffrom.keras.models.load_model('marital_status_cnn_model.model')
encoded_docs = [one_hot(d.lower(), vocab_size) for d in docs]
# pad documents to a max length of 4 words
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
result = loaded_model.predict(padded_docs)
ind = 0
for row in result:
    if(row[0]<0.5):
        out = 0
    else:
        out = 1
    ind+=1

print(docs[0]," - ",categories[out])

My boyfriend is a talented photographer working in Stanford university  -  Single
